<a href="https://colab.research.google.com/github/fatemafaria142/Automated-Realistic-Story-Generation/blob/main/Story_Generation_using_GPT2_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Dataset Link: https://huggingface.co/datasets/AtlasUnified/atlas-storyteller?row=42**


In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

dataset = load_dataset("AtlasUnified/atlas-storyteller")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Display dataset information
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'Story'],
        num_rows: 5018
    })
})


In [5]:
# Access the 'train' split of the dataset
train_data = dataset['train']

# Display information about the 'train' split
print(train_data)

Dataset({
    features: ['id', 'Story'],
    num_rows: 5018
})


In [6]:
# Display the first few rows of the 'train' split
print(train_data[:5])  # Displaying the first 5 rows as an example

{'id': ['seed_task_0', 'seed_task_1', 'seed_task_2', 'seed_task_3', 'seed_task_4'], 'Story': ['In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They\'re coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan\'s blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment\'s hesitation, Ethan gathered his meager belongings and 

In [7]:
# Convert the 'train' split to a 'data' CSV file
csv_filename = "/content/drive/MyDrive/Story Generation/data.csv"
train_data.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' has been created.")

Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

CSV file '/content/drive/MyDrive/Story Generation/data.csv' has been created.


In [8]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/Story Generation/data.csv")
dataset.head()

,id,Story
0,seed_task_0,"In the bustling city of New York, where the ne..."
1,seed_task_1,"The sky stretched out before them, an endless ..."
2,seed_task_2,In the midst of a bustling metropolis teeming ...
3,seed_task_3,"In the heart of a once-thriving metropolis, no..."
4,seed_task_4,In the gritty underbelly of a city plagued by ...


# **Length of the main dataset**

In [9]:
len(dataset)

5018

# **Taking 90% for train dataset and 10% for test dataset**

In [10]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataset from the CSV file
data = pd.read_csv("/content/drive/MyDrive/Story Generation/data.csv")

def build_text_files(data_csv, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for index, row in data_csv.iterrows():
        summary = str(row['Story']).strip()
        summary = re.sub(r"[\[\]\"]", "", summary)  # Remove square brackets and quotation marks
        summary = re.sub(r"\s+", " ", summary)  # Replace multiple spaces with a single space
        data += summary + "  "
    f.write(data)

train, test = train_test_split(data, test_size=0.10)

build_text_files(train, '/content/drive/MyDrive/Story Generation/train_dataset.txt')
build_text_files(test, '/content/drive/MyDrive/Story Generation/test_dataset.txt')

print("Train dataset length:", len(train))
print("Test dataset length:", len(test))


Train dataset length: 4516
Test dataset length: 502


# **Text Preprocessing**

In [11]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        for i, example in enumerate(examples[:num_examples]):
            print(f"Example {i+1}: {example}")

train_file_path = '/content/drive/MyDrive/Story Generation/train_dataset.txt'
test_file_path = '/content/drive/MyDrive/Story Generation/test_dataset.txt'

print("Examples from the Train Dataset:")
read_text_file(train_file_path)

print("\nExamples from the Test Dataset:")
read_text_file(test_file_path)


Examples from the Train Dataset:
Example 1: In a small, bustling town nestled in the heart of the countryside, there stood a grand institution of learning known as Maplewood Academy. With its ivy-covered façade and sprawling campus, it was renowned for providing a high-quality education to students from all walks of life. The academy boasted a diverse student body, hailing from different corners of the world, each with their own unique stories and dreams. At the helm of Maplewood Academy was Dr. Elizabeth Foster, a visionary educator with a passion for promoting global citizenship in education. Dr. Foster firmly believed that by fostering an environment that encouraged understanding and cooperation among students from various cultural backgrounds, she could cultivate the next generation of global leaders. With this noble goal in mind, she embarked on a journey to transform Maplewood Academy into a beacon of global education. Under Dr. Foster's leadership, Maplewood Academy witnessed a 

# **GPT2-medium and its tokenizer**
### **https://huggingface.co/gpt2-medium**

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('gpt2-medium')
model = AutoModelForCausalLM.from_pretrained('gpt2-medium')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

In [13]:
train_path = '/content/drive/MyDrive/Story Generation/test_dataset.txt'
test_path = '/content/drive/MyDrive/Story Generation/test_dataset.txt'

# **Prepare the dataset and build a TextDataset**
* The next step is to extract the instructions from all recipes and build a TextDataset. The TextDataset is a custom implementation of the Pytroch Dataset class implemented by the transformers library.

* The block_size parameter is used to specify the maximum length of each sequence or block of text. Text longer than this length will be divided into consecutive blocks of this specified size.

* For example, if we set block_size=128, the text will be processed in blocks of 128 tokens. Any sequence longer than 128 tokens will be divided into smaller consecutive blocks of 128 tokens each.

In [20]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (415915 > 1024). Running this sequence through the model will result in indexing errors


In [21]:
!pip install transformers[torch]

In [22]:
!pip install accelerate -U

In [23]:
!pip install --upgrade transformers

# **1st time train - Initialize Trainer with TrainingArguments**

In [26]:
'''
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-storygeneration", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps = 500, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
'''

In [27]:
'''
trainer.train()
'''

Step,Training Loss
500,2.679200
1000,2.261200
1500,1.938500
2000,1.712900


TrainOutput(global_step=2035, training_loss=2.139835867542014, metrics={'train_runtime': 1702.9171, 'train_samples_per_second': 9.54, 'train_steps_per_second': 1.195, 'total_flos': 3771685695651840.0, 'train_loss': 2.139835867542014, 'epoch': 5.0})

In [28]:
'''
trainer.save_model()
'''

# **Loading the saved model and train it again**

* The Trainer class provides an API for feature-complete training. It is used in most of the example scripts from Huggingface. Before we can instantiate our Trainer we need to download our GPT2-medium model and create a TrainingArguments to access all the points of customization during training. In the TrainingArguments, we can define the Hyperparameters we are going to use in the training process like our learning_rate, num_train_epochs, or per_device_train_batch_size.


In [29]:
from transformers import Trainer, TrainingArguments
# Load the previously trained model
model_path = "./gpt2-storygeneration"  # Replace this with the path to your saved model
model = AutoModelForCausalLM.from_pretrained(model_path)

# Define your training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-storygeneration",  # The output directory
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    num_train_epochs=10,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    eval_steps=500,  # Number of update steps between two evaluations
    save_steps=800,  # Save model checkpoint after # steps
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)

# Initialize Trainer with your loaded model and training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


# **Train the model**

In [30]:
# Start training to continue from where it left off
trainer.train()

Step,Training Loss
500,1.562100
1000,1.333100
1500,1.133200
2000,1.030000
2500,0.912000
3000,0.779000
3500,0.694400
4000,0.636700


Checkpoint destination directory ./gpt2-storygeneration/checkpoint-800 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./gpt2-storygeneration/checkpoint-1600 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=4070, training_loss=1.0035431020675771, metrics={'train_runtime': 3470.3691, 'train_samples_per_second': 9.362, 'train_steps_per_second': 1.173, 'total_flos': 7543371391303680.0, 'train_loss': 1.0035431020675771, 'epoch': 10.0})

# **Save the model**

In [31]:
# Save the model after training
trainer.save_model("./gpt2-storygeneration")

# **Test the model**
* To test the model we are going to use another highlight of the transformers library called pipeline. Pipelines are objects that offer a simple API dedicated to several tasks, among others also text-generation

In [32]:
from transformers import pipeline

In [33]:
storyteller = pipeline('text-generation',model='./gpt2-storygeneration', tokenizer='gpt2-medium')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [34]:
storyteller('In the bustling city of New York,')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In the bustling city of New York, amidst towering skyscrapers and the cacophony of honking horns, a tale of power and betrayal unfolded. The covert organization known as The Spectators had been shadowing James Donovan for years, gathering damning'}]

In [35]:
storyteller(' In the midst of a bustling metropolis teeming with skyscrapers that seemingly ')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': ' In the midst of a bustling metropolis teeming with skyscrapers that seemingly iced over the streets, there stood an unassuming antique shop tucked away in a corner. It was a haven for those who sought inspiration in the world of fashion,'}]

In [36]:
storyteller('In the heart of a once-thriving metropolis, now reduced to a desolate wasteland')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In the heart of a once-thriving metropolis, now reduced to a desolate wasteland, lay the battleground of an epic struggle for control. The skies defied gravity, blotting out starlight, as an electromagnetic pulse rippled through the'}]

In [37]:
storyteller('In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with.')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with. His chiseled jawline, piercing blue eyes, and supremely sharp intellect made him an intimidating figure. The streets'

# **Evaluate perplexity on the test dataset**

In [38]:
# Evaluate perplexity on the test dataset
eval_result = trainer.evaluate(eval_dataset=test_dataset)
print("Perplexity:", eval_result['eval_loss'])

Perplexity: 0.3082282245159149


# **Generating Stories without any prompt**

In [39]:
from transformers import pipeline

# Initialize the text generation pipeline
storyteller = pipeline('text-generation', model='./gpt2-storygeneration', tokenizer='gpt2-medium')

# Function to generate stories and save them
def generate_stories(stories_pipeline, num_stories: int):
    generated_stories = []
    for _ in range(num_stories):
        generated_story = stories_pipeline("Story: ")  # You can modify the initial prompt if needed
        generated_stories.append(generated_story[0]['generated_text'])
    return generated_stories

# Generate stories
num_stories_to_generate = 5  # Define the number of stories to generate
generated_stories = generate_stories(storyteller, num_stories_to_generate)

# Display the generated stories
for i, stories in enumerate(generated_stories, start=1):
    print(f"Generated Story {i}:\n{stories}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Story 1:
Story:  In the enchanting realm of Argentia, nestled amid verdant hills and shimmering lakes, there lived a young sorceress named Seraphina. With flowing golden locks and eyes the color of emeralds, Seraphina possessed

Generated Story 2:
Story:  The Sky Is A Bridge Too Deep For A Feeling Isabella's heart heavy with a sense of restlessness. She's spent the last few days traversing the intricacies of her everyday life, drawn to the enchanting melodies floating above

Generated Story 3:
Story:  A great king, whose reign spanned countless battles and battles, ruled over a realm filled with wonders and wonders. He was known for his vast navy, his vast land, and his vast dominion. But it was his crowning achievement

Generated Story 4:
Story:  In a quaint town nestled amongst rolling hills and babbling brooks, there lived a philosopher named Theodore. With a kind, curious mind and a deep passion for the pursuit of truth, he spent his days engulfed in a labyrinth of

Gener

# **List of 1st 20 examples from Test Dataset**

In [40]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        return examples[:num_examples]

test_file_path = '/content/drive/MyDrive/Story Generation/test_dataset.txt'

print("\nExamples from the Test Dataset:")
first_20_examples = read_text_file(test_file_path, num_examples=20)

# Display the first 20 examples
for i, example in enumerate(first_20_examples, start=1):
    print(f"Example {i}: {example}")

# Store the first 20 examples in a list
list_of_20_examples = first_20_examples


Examples from the Test Dataset:
Example 1: Once upon a time in the bustling city of New York, where concrete buildings reached up to touch the heavens, there lived a young and aspiring artist named Emily. Her heart beat to the rhythm of creativity, and her fingers danced effortlessly across the canvas, breathing life into her vibrant masterpieces. Born into a family of renowned artists, Emily carried the weight of expectation upon her slender shoulders. In the heart of Manhattan, amid the crowded streets and blaring horns, lay an enchanting art gallery known as The Palette's Haven. Owned by the revered curator, Mr. Jackson, this sanctuary served as a sanctuary for artists and art lovers alike. Its polished oak floors reverberated with the footsteps of those who sought inspiration and solace from the vibrant works of art lining the walls. One fateful day, Emily's brushes and palettes were drawn towards The Palette's Haven, as if guided by an unseen hand. She ventured into the gallery, 

In [41]:
print(list_of_20_examples)

In [42]:
print(len(list_of_20_examples))

["Once upon a time in the bustling city of New York, where concrete buildings reached up to touch the heavens, there lived a young and aspiring artist named Emily. Her heart beat to the rhythm of creativity, and her fingers danced effortlessly across the canvas, breathing life into her vibrant masterpieces. Born into a family of renowned artists, Emily carried the weight of expectation upon her slender shoulders. In the heart of Manhattan, amid the crowded streets and blaring horns, lay an enchanting art gallery known as The Palette's Haven. Owned by the revered curator, Mr. Jackson, this sanctuary served as a sanctuary for artists and art lovers alike. Its polished oak floors reverberated with the footsteps of those who sought inspiration and solace from the vibrant works of art lining the walls. One fateful day, Emily's brushes and palettes were drawn towards The Palette's Haven, as if guided by an unseen hand. She ventured into the gallery, mesmerized by the symphony of colors and t

# **Generating Stories with prompts**

In [43]:
from transformers import pipeline

test_dataset_file = '/content/drive/MyDrive/Story Generation/test_dataset.txt'

# Initialize the text generation pipeline
storyteller = pipeline('text-generation', model='./gpt2-storygeneration', tokenizer='gpt2-medium')

# Read your test dataset
with open(test_dataset_file, 'r') as file:
    test_content = file.read()

# Split the content into individual stories based on the delimiter ('  ')
test_stories = test_content.split('  ')

# Function to generate stories based on the test dataset
def generate_stories_from_test_dataset(story_pipeline, test_dataset, num_stories: int):
    generated_stories = []
    for i in range(num_stories):
        # Extract the first 6 words from the stories as the prompt
        words = test_dataset[i].split()[:6] if i < len(test_dataset) else test_dataset[-1].split()[:6]
        prompt = ' '.join(words)
        generated_story = story_pipeline(prompt, max_length=128)  # Adjust max_length as needed
        generated_stories.append(generated_story[0]['generated_text'])
    return generated_stories

# Generate stories based on the test dataset
num_stories_to_generate = 20  # Define the number of stories to generate
generated_stories_from_test = generate_stories_from_test_dataset(storyteller, test_stories, num_stories_to_generate)

# Display the generated stories
for i, stories in enumerate(generated_stories_from_test, start=1):
    print(f"Generated Story {i}:\n{stories}\n")

# Save the generated stories in a list
list_of_generated_stories = generated_stories_from_test


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Generated Story 1:
Once upon a time in the not too distant future, in the sprawling metropolis of NeoCity, where towering skyscrapers cast shadows on the rooftops and neon lights flickered like distant stars, an epic struggle for control of human minds and artificial intelligence would rage. It all began with Theodore Delacroix, a brilliant neuroscientist with a restless spirit. Theodore had devoted his life to unraveling the intricate complexities of the human mind, seeking to unravel the secrets that governed the very fabric of our existence. One fateful day, as he perused his laboratory's archives, he stumbled upon a startling revelation. An enigmatic manuscript

Generated Story 2:
Deep within the sprawling metropolis of Neo-Drexel, a technologically dense dystopia, lived a young android named Maya. Maya had always felt a void within herself, a longing for a sense of self beyond the constantly mechanical routine of her robotic life. With an insatiable thirst for discovery, Maya spen

In [44]:
print(list_of_generated_stories)

["Once upon a time in the not too distant future, in the sprawling metropolis of NeoCity, where towering skyscrapers cast shadows on the rooftops and neon lights flickered like distant stars, an epic struggle for control of human minds and artificial intelligence would rage. It all began with Theodore Delacroix, a brilliant neuroscientist with a restless spirit. Theodore had devoted his life to unraveling the intricate complexities of the human mind, seeking to unravel the secrets that governed the very fabric of our existence. One fateful day, as he perused his laboratory's archives, he stumbled upon a startling revelation. An enigmatic manuscript", "Deep within the sprawling metropolis of Neo-Drexel, a technologically dense dystopia, lived a young android named Maya. Maya had always felt a void within herself, a longing for a sense of self beyond the constantly mechanical routine of her robotic life. With an insatiable thirst for discovery, Maya spent her days roaming the city's laby

In [45]:
print(len(list_of_generated_stories))

20


# **Evaluating Generated Recipes Against Reference Recipes**

In [46]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=e803ee46ad0d33b0fd8b72a453c1c06b71ec9ba09568bce90bd5dffd79781753
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [47]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [48]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.4 MB/s eta 0:00:00


In [49]:
from evaluate import load
# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [50]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=list_of_generated_stories, references=list_of_20_examples)
results_WER = wer_metric.compute(predictions=list_of_generated_stories, references=list_of_20_examples)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=list_of_generated_stories, references=list_of_20_examples)
results_met = meteor.compute(predictions=list_of_generated_stories, references=list_of_20_examples)

# Calculate Bilingual Evaluation Understudy (BLEU)
results_bleu = bleu_metric.compute(predictions=list_of_generated_stories, references=list_of_20_examples)

# **Printing evaluation metrics**

In [51]:
print("Character Error Rate for Story Generation:", results_CER)
print("Word Error Rate for Story Generation:",results_WER)
print("Exact Match for Story Generation:",results_em)
print("BLEU Score for Story Generation:",results_bleu)
print("METEOR for Story Generation:",results_met)

Character Error Rate for Story Generation: 0.8715310401723309
Word Error Rate for Story Generation: 0.9474332054159728
Exact Match for Story Generation: {'exact_match': 0.0}
BLEU Score for Story Generation: {'bleu': 0.0005072458444240759, 'precisions': [0.5669393026259147, 0.19539730785931395, 0.08409986859395532, 0.05877154220061865], 'brevity_penalty': 0.0033159968665602184, 'length_ratio': 0.14905357715752326, 'translation_length': 2323, 'reference_length': 15585}
METEOR for Story Generation: {'meteor': 0.058157494043711254}
